In [1]:
import boto3
import pandas as pd
import numpy as np
import sys
from io import StringIO
import surprise
from surprise import accuracy
from sklearn.model_selection import train_test_split
from surprise import SVD, NMF, KNNBaseline


In [2]:
sys.path.insert(1, '../src')
from Collaborative_recommender import CollaborativeRecommender
from Content_recommender import ContentRecommender
from NLP import nlp_process

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext autoreload

In [5]:
s3 = boto3.client('s3')

In [6]:
df_all_15 = pd.read_csv("s3://recommender-system-amazon/df_all_15_text.csv")

### Prepare required dataframe for the hybrid recommender 

In [7]:
df_items_text = df_all_15.groupby(['asin', 'title','title_nlp','image_x'])['item_text'].apply(lambda x: ' '.join(x)).reset_index()

In [8]:
df_collab = df_all_15[['reviewerID',"asin","overall"]]

In [9]:
weighted_vector_text = pd.read_csv("s3://recommender-system-amazon/weighted_vector_text.csv")

In [12]:
weighted_vector_text.drop('Unnamed: 0',axis=1, inplace = True)

In [14]:
weighted_vector_text.rename(index=df_items_text.asin,inplace=True)

### Collaborative filter

In [17]:
reader = surprise.Reader(rating_scale=(1,5))
recom_colab = CollaborativeRecommender(df_collab,reader,NMF)
param_grid = {'n_factors': [8,15,23,30,37,42], 'n_epochs': [5, 10]}
recom_colab.grid(param_grid,cv=3)
recom_colab.fit()

In [21]:
df_pred, acc = recom_colab.pred()
UM = recom_colab.utility_matrix()

RMSE: 0.6924


In [22]:
UI = "AZZG39P6RDCHL"
recom_colab.get_top_n(UI)

,predicted_rating
iid,
B001ASUH4A,5.000000
B004TBMUMM,5.000000
B00AT6HZGM,5.000000
B00I77PNUA,5.000000
B00QBLAQCE,5.000000
B00GU3K5AQ,4.729353
B000P3WQ86,4.391842
B01D8WEH14,4.303853
7245456275,NaN
